Collaborator: 
    Zachary Chommala
    Alex Ho
    Bao Nguyen

# CSCI 3155: Assignment 5

Topics Covered: Operations on inductive definitions, building an interpreter, big step operational semantics and using map, foldLeft and filter functions to replace loops.

__Jonathan Phouminh__

In [4]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"Tests Passed (1 point)")
    else print(s"Tests Passed ($points points)") 
}

defined function passed

## Problem 1: Manipulating ASTs, Inference Rules (30 points)

### 1A: Derivatives of Expressions (20 points)

We have defined a grammar for arithmetic expressions in our class notes. 

$$\begin{array}{rcll}
\textbf{Expr} & \rightarrow & Const(\textbf{Double}) \\
& |  & Ident(\textbf{String}) \\
& | & Plus( \textbf{Expr}, \textbf{Expr})  \\
& | & Minus( \textbf{Expr}, \textbf{Expr}) \\
& | & Mult(\textbf{Expr}, \textbf{Expr}) \\
& | & Div(\textbf{Expr}, \textbf{Expr}) \\
& | & Exp(\textbf{Expr}) \\
& | & Sine(\textbf{Expr}) \\
& | & Cosine(\textbf{Expr}) \\\\
\textbf{Double} & \rightarrow & \text{all double precision numbers in Scala}\\
\textbf{String} & \rightarrow & \text{all scala strings}\\
\end{array}$$

For this problem, you will be writing an _automatic differentiation_ method that, given an expression `e` which involves just a single identifier `x` (no need to check this fact), will return an expression for `de/dx`, the derivative of `e` with respect to `x`.

Eg., `e = Sine(Mult(Ident("x"), Const(2.0)))` should return `Mult(Const(2.0), Cosine(  Mult(Ident("x"), Const(2.0)))`.  In plain math, $\frac{d \sin(2x)}{dx} = 2 \cos( 2x) $

We will write down the inference rules for derivative, as follows.

A rule for constants ($\frac{dc}{dx} = 0, c \in \mathbb{R}$)

$\begin{array}{c}
\\
\hline 
\text{derivative}( \texttt{Const(f)} , x) = \texttt{Const(0.0)} \\
\end{array} \mathbf{(Constant)}$      

A rule for identifiers $\frac{dx}{dx} = 1, \frac{dy}{dx} = 0$ for $y \not= x$.

$\begin{array}{c}
\\
\hline 
\text{derivative}( \texttt{Ident(s)} , x) = \left\{ \begin{array}{ll} \texttt{Const(1.0)} & x == s \\
\texttt{Const(0.0)} & \text{otherwise} \end{array} \right.\\
\end{array} \mathbf{(Identifier)}  \;\;\;
$

A rule for plus $\frac{d}{dx} (e_1 + e_2) = \frac{de_1}{dx} + \frac{de_2}{dx}$.

$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1},\;\;\text{derivative}(\texttt{e2}, x) = \texttt{f2}\\
\hline
\text{derivative}(\texttt{Plus(e1, e2)}, x) = \texttt{Plus(f1, f2)} \\
\end{array} \mathbf{(Plus)} $

A rule for multiplication: $\frac{d}{dx} (e_1 e_2) = e_2 \frac{de_1}{dx} + e_1 \frac{de_2}{dx}$.

$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1},\;\;\text{derivative}(\texttt{e2}, x) = \texttt{f2}\\
\hline
\text{derivative}(\texttt{Mult(e1, e2)}, x) = \texttt{Plus(Mult(f1, e2), Mult(f2, e1))} \\
\end{array} \mathbf{(Mult)} $

A rule for division $\frac{d}{dx} \left(\frac{e_1}{e_2}\right) = \frac{\frac{de_1}{dx}}{e_2} - \frac{e_1 \frac{d e_2}{dx}}{e_2^2}$

$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1},\;\;\text{derivative}(\texttt{e2}, x) = \texttt{f2}\\
\hline
\text{derivative}(\texttt{Div(e1, e2)}, x) = \texttt{Minus(Div(f1, e2), Div(Mult(e1, f2), Mult(e2, e2)))} \\
\end{array} \mathbf{(Div)} $

A rule for exponentiation $\frac{d}{dx} \left(e^{e_1}\right) = e^{e_1} \frac{de_1}{dx}$

$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1}\\
\hline
\text{derivative}(\texttt{Exp(e1)}, x) = \texttt{Mult(Exp(e1), f1)} \\
\end{array} \mathbf{(Exp)} $

The rules for $\mathbf{Minus}$, $\mathbf{Sine}$, and $\mathbf{Cosine}$ are left for you to write.
You do not need to write your solution for these rules in this notebook, __but__, you do need to give them in code below.
(Don't forget about the chain rule!)

In [2]:
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr
case class Exp(e: Expr) extends Expr
// We will skip over Log(e: Expr)

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Sine
defined class Cosine
defined class Exp

In [3]:
/* test cases
// should return true arbitraily
    val constant = Const(1000.0)
    //println("-->", checkWithRespect(constant,identifier))
    
    
    // testing out the single identifiers
    val singleIdentifer = Ident("x") // this returned true, which is correct
    val singleIdentifer2 = Ident("y") // this returned false, which is correct
    //println("-->", checkWithRespect(singleIdentifer,identifier))
    
    
    // testing out the Exp(..) identifiers
    val Exponent1 = checkWithRespect(Exp(Const(1.0)),identifier)
    val Exponent2 = checkWithRespect(Exp(Ident("x")),identifier)
    
    // testing out the Plus(e1,e2) should be able to do it on a general level of using only ident const exp
    val add = Plus(Exp(Const(1.0)), Ident("x"))
    val add1 = Plus(Exp(Const(1.0)), Ident("y"))
    
//     println("TEST1: ", checkWithRespect(add, identifier))
//     println("TEST2: ", checkWithRespect(add1, identifier))    
    
    
    // testing out the Minus
    val minus = Minus(Exp(Const(1.0)), Ident("x"))
    val minus1 = Minus(Exp(Const(1.0)), Ident("y"))
    //println("TEST1: ", checkWithRespect(minus, identifier))
    //println("TEST2: ", checkWithRespect(minus1, identifier))    
    
    // Testing out Div 
    val div = Div(Exp(Const(1.0)), Ident("x"))
    val div1 = Div(Exp(Const(1.0)), Ident("y"))
    //println("TEST1: ", checkWithRespect(div, identifier))
    //println("TEST2: ", checkWithRespect(div1, identifier))  
    
    // Testing out Multiplication
    val mul = Mult(Exp(Const(1.0)), Ident("x"))
    val mul1 = Mult(Exp(Const(1.0)), Ident("y"))
    println("TEST1: ", checkWithRespect(mul, identifier))
    println("TEST2: ", checkWithRespect(mul1, identifier)) 
*/
/*
    Function takes in an expression that needs to be evaluated ( derivative must be calculated ) 
    and is also given in the identifier that it is going to be evaulated with respect to.

        - first things first would most likely to make sure that the expression is with respect to the identifer
        - if its not then just return the expression 
        
        - otherwise
            - must evaluate the derivative of Plus
                just check if it is with respect to the string first
                if so then get the derivative of its to compnents and perform operation
            - derivative of Minus
                just check if it is with respect to the string first
                if so then get the derivative of its to compnents and perform operation
            - Derivative Cosine
                -sin(...) <-- chain rule might apply here
            - Derivative Sine
                cos(...) <-- chain rule might apply here
            - Derivative Div
                (low)dhigh - high(dlow) / low**2
            - Derivative of Exp
                power rule
            - Derivative of Mult
            
            Plus(Cosine(1x), Sine(x))  => Plus( -Sin(2x) )
*/

/*
------------------------------------------------------------------------------------------------------------------              
    Function will take in an expression that needs to be checked if its with respect to its identifer
    function then returns true or false based on if the expression is with respect to its identifier. 
    
    how this function will work. 
        essentially this function will need to keep breaking down the expression until it is 
        of the type, 'Ident(..)' , if we break down the expression all the way, and it is not, ident
        then we just return false UNLESS it is of type const because its can be handled.
*/
def checkWithRespect(e: Expr, x:String): Boolean = e match{
    // if its a const just return true  
    case Const(_) => true
    // unpack the identifier and if it is of the same same type x then return true 
    case Ident(y) => {
        if ( y == x ){
            true
        }else{
            false
        }
    }
    // must unpack e1 to see what the true expression of e1 actually is 
    case Exp(e1) => {
        checkWithRespect(e1,x)
    }
    case Cosine(e1) => {
        checkWithRespect(e1,x)
    }
    case Sine(e1) => {
        checkWithRespect(e1,x)
    }
    case Plus(e1,e2) => {
        val leftExpression = checkWithRespect(e1, x)
        val rightExpression = checkWithRespect(e2, x)
        // if both of the expressions evaluate to true, then we can proceed to find the derivative
        if (leftExpression == rightExpression){
            true
        }else{
            throw new IllegalArgumentException(s"Expression ${e} is not with respect identifier ${x}")
            false
        }
    }
    case Minus(e1,e2) => {
        val leftExpression = checkWithRespect(e1, x)
        val rightExpression = checkWithRespect(e2, x)
        // if both of the expressions evaluate to true, then we can proceed to find the derivative
        if (leftExpression == rightExpression){
            true
        }else{
            throw new IllegalArgumentException(s"Expression ${e} is not with respect identifier ${x}")
            false
        }
    }
    case Div(e1,e2) => {
        val leftExpression = checkWithRespect(e1, x)
        val rightExpression = checkWithRespect(e2, x)
        // if both of the expressions evaluate to true, then we can proceed to find the derivative
        if (leftExpression == rightExpression){
            true
        }else{
            throw new IllegalArgumentException(s"Expression ${e} is not with respect identifier ${x}")
            false
        }
    }
    case Mult(e1,e2) => {
        val leftExpression = checkWithRespect(e1, x)
        val rightExpression = checkWithRespect(e2, x)
        // if both of the expressions evaluate to true, then we can proceed to find the derivative
        if (leftExpression == rightExpression){
            true
        }else{
            throw new IllegalArgumentException(s"Expression ${e} is not with respect identifier ${x}")
            false
        }
    }
}

// misread the directions, thought you had to check the expression type. 


defined function checkWithRespect

Write a function `derivativeExpr` that calculates the derivative of an expression
`e` w.r.t a given identifier as a string `x`.

In [4]:

// Up to this point you are now able to check if the expression you are is in respect to the identifier x 
// now you jsut need to start coming up with the actual derivatives
// helper function that will return the expression of the calculated derivative
def derivativeHelper(e: Expr, x: String): Expr = e match{
    // the derivative of a constant is equal to 0 
    case Const(e1) =>  Const(0.0)
    
    // derivative of a single identifer is 1 ( dy/dx(x) == 1 )
    case Ident(y) => {
        if ( y == x ){
            Const(1.0)
        }else{
            Const(0.0)
        }
    }

    // Derivative of Sine(..) is Cos(..), Sine is only defined to take in one input in our case
    case Sine(e1) => e1 match{
        //case Ident(_) => Cosine(e1)
        //case Const(_) => Cosine(derivativeHelper(e2,x))
        //case _ => Cosine(derivativeHelper(e1, x))
        case Ident(e1) => Mult(Cosine(Ident(e1)), derivativeHelper(Ident(e1),x))
        case Const(e1) => Mult(Cosine(Const(e1)), derivativeHelper(Const(e1),x))
        case _ =>  Mult(Cosine(e1), derivativeHelper(e1,x))
    }
    
     // Derivative of Sine(..) is Cos(..), Sine is only defined to take in one input in our case
    case Cosine(e1) => e1 match{
        // -1x == Mult(Const(-1.0),Ident(x))
        case Ident(_) => Mult(Const(-1),Sine(e1))
        case Const(_) => Mult(Const(-1),derivativeHelper(e1,x))
        case _ => Mult(Const(-1),derivativeHelper(e1, x))
    } 
    
    case Exp(e1) => {
        val f1 = derivativeHelper(e1,x)
        Mult(Exp(e1),f1)
    }
    
    case Mult(e1,e2) => {
        val der1 = derivativeHelper(e1,x)
        val der2 = derivativeHelper(e2,x)
        Plus(Mult(der1,e2),Mult(der2,e1))
    }
    
    case Div(e1,e2) => {
        val f1 = derivativeHelper(e1,x)
        val f2 = derivativeHelper(e2,x) 
        
        println("F1: ", f1, "F2: ", f2)
        println()
        println()
        Minus(Div(f1,e2),Div( Mult(e1,f2), Mult(e2,e2)))
    }
    
    // Derivative of plus is equal to derivative of left value plus derivative of right value
    case Plus(e1,e2) => {
        Plus(derivativeHelper(e1, x),derivativeHelper(e2, x))
    }
    
    // Derivative of minus is equal to the derivative of the left value minus derivative of left value
    case Minus(e1,e2) => {
        Minus(derivativeHelper(e1, x),derivativeHelper(e2, x))
    }
}

def derivativeExpr(e: Expr, x: String): Expr = {
    // if we pass this function then we know we can evaluate it, otherwise
    // itll just throw an error of its own 
    checkWithRespect(e,x)
    // going to pass this along to a helper because you are most likely going to have to 
    // construct the returned expression recursively
    val returnedExpression = derivativeHelper(e, x)
    //println("ORIGINAL EXPRESSION: ",e)
    //println("MY EXPRESSION: ", returnedExpression)
    returnedExpression
}
    



defined function derivativeHelper
defined function derivativeExpr

In [5]:
// BEGIN TESTS

// TEST HELPERS
def evalExpr (e: Expr, env: Map[String, Double]): Double = e match {
    case Const (f) => f
    case Ident (str) => { if (env.contains(str)){
                                env(str)
                            } else {
        throw new IllegalArgumentException(s"Environment does not contain mapping for $str")
    }
                        }
    case Plus(e1, e2) => {
        (evalExpr(e1, env)) + (evalExpr(e2, env))
    }
    
    case Minus(e1, e2) => {
        (evalExpr(e1, env)) - (evalExpr(e2, env))
    }
    
    case Mult(e1, e2) => {
        (evalExpr(e1, env)) * (evalExpr(e2, env))
    }
    
    case Div(e1, e2) => {
        (evalExpr(e1, env)) / (evalExpr(e2, env))
    }
    
    case Exp(e) => math.exp( evalExpr(e, env))
    
    case Sine(e) => math.sin( evalExpr(e, env))
    
    case Cosine(e) => math.cos(evalExpr(e, env))
}

def testExpressions(exp: Expr, deriv_expected: Expr, testVals: List[Double]): Boolean = {
    val tol: Double = 1E-06
    val deriv_act = derivativeExpr(exp, "x")
    testVals forall { 
            x => {
              val res = math.abs( evalExpr(deriv_act, Map("x"-> x)) - evalExpr(deriv_expected, Map("x" -> x)) ) <= tol
              if (!res) { println(s"Failed at $x")}
              res
            }
    }
}

val allVals = List(-5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -1.9, -1.4, -1.0, -0.5, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0)

defined function evalExpr
defined function testExpressions
allVals: List[Double] = List(
  -5.0,
  -4.5,
  -4.0,
  -3.5,
  -3.0,
  -2.5,
  -1.9,
  -1.4,
  -1.0,
  -0.5,
  0.1,
  0.5,
  1.0,
  1.5,
  2.0,
  2.5,
  3.0,
  3.5,
  4.0,
  4.5,
  5.0
)

In [6]:
val e1 = Plus(Ident("x"), Const(2.0))
assert(testExpressions(e1, Const(1.0), allVals ), s"Test 1 Failed -- Input: $e1 ")

passed(5)

Tests Passed (5 points)

e1: Plus = Plus(Ident("x"), Const(2.0))

In [7]:
val e2 = Plus(Cosine(Ident("x")), Sine(Ident("x")))
val ed2 = Minus(Cosine(Ident("x")), Sine(Ident("x")))
assert(testExpressions(e2, ed2, allVals), s"Test 2 Failed: Input is $e2")

passed(5)

Tests Passed (5 points)

e2: Plus = Plus(Cosine(Ident("x")), Sine(Ident("x")))
ed2: Minus = Minus(Cosine(Ident("x")), Sine(Ident("x")))

In [8]:
val x = Ident("x")
val e3 = Exp(Mult(x, x))
val ed3 = Mult(Mult(Const(2.0), x), e3)
assert(testExpressions(e3, ed3, allVals), s"Test 3 Failed: Input is $e3")

passed(5)

Tests Passed (5 points)

x: Ident = Ident("x")
e3: Exp = Exp(Mult(Ident("x"), Ident("x")))
ed3: Mult = Mult(
  Mult(Const(2.0), Ident("x")),
  Exp(Mult(Ident("x"), Ident("x")))
)

In [9]:
val e4 = Div(x, Plus(x, Const(2.0)))
val ed4 = Div(Const(2.0), Mult(Plus(x, Const(2.0)), Plus(x, Const(2.0))) )
assert(testExpressions(e4, ed4, allVals), s"Test 4 Failed: Input is $e4")


val e5 = Sine(Mult(Exp(Minus( Cosine(Div(x,x)), Cosine(Const(1.0)) )), x))
val ed5 = Cosine(x)
assert(testExpressions(e5, ed5, allVals), s"Test 5 Failed: Input is $e5")

passed(5)
// END TESTS

(F1: ,Const(1.0),F2: ,Plus(Const(1.0),Const(0.0)))


(F1: ,Const(1.0),F2: ,Const(1.0))


Tests Passed (5 points)

e4: Div = Div(Ident("x"), Plus(Ident("x"), Const(2.0)))
ed4: Div = Div(
  Const(2.0),
  Mult(Plus(Ident("x"), Const(2.0)), Plus(Ident("x"), Const(2.0)))
)
e5: Sine = Sine(
  Mult(
    Exp(Minus(Cosine(Div(Ident("x"), Ident("x"))), Cosine(Const(1.0)))),
    Ident("x")
  )
)
ed5: Cosine = Cosine(Ident("x"))

### 1B: Newton's Algorithm Reloaded (10 points)

Let's revisit problem 5 from assignment 1. Back in assignment 1, we hard coded the
expression and its derivative. Here, we will use our `Expr` abstract syntax to 
define expressions and use the function you wrote in 2A to compute derivatives.
You may also use the `evalExpr` function provided below.

Newton invented the Newton-Raphson method for solving an equation. 
We are going to ask you to write some code to solve equations.

`solveEquation(e: Expr, x0: Double, maxIters: Int = 1000)`

Assume that the input expression has involves just one variable "x".

To solve an equation of the form

$$ x^2 - 3x + 2 == 0$$

with initial guess at the solution: say $$x_0 = 4.5$$,

We will input `val e = Plus(Minus(Mult(Ident("x"), Ident("x")), Mult(Const(3.0), Ident("x"))), Const(2.0))`
into the function

`solveEquation( e, 1.5, 1000)`

Each time we have the $i^{th}$ guess $x_i$, we update it as

$$ x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)} $$

For our equation, $f(x) = x^2 - 3x +2$ and $f'(x) = 2 x - 3$ ( $f'$ is the derivative of $f$).

Thus, our update equation is 
$$ x_{i+1} = x_i - \frac{x_i^2 - 3 x_i + 2}{2 x_i - 3}$$.

We stop whenever $|f(x_i)| \leq 10^{-8}$ : i.e, we are very close to a root of the function
_or_ $ i \geq \text{maxIters}$. 

Gory details are here:
http://www.math.ubc.ca/~anstee/math104/newtonmethod.pdf



In [10]:
def evalExpr (e: Expr, env: Map[String, Double]): Double = e match {
    case Const (f) => f
    case Ident (str) => { if (env.contains(str)){
                                env(str)
                            } else {
        throw new IllegalArgumentException(s"Environment does not contain mapping for $str")
    }
                        }
    case Plus(e1, e2) => {
        (evalExpr(e1, env)) + (evalExpr(e2, env))
    }
    
    case Minus(e1, e2) => {
        (evalExpr(e1, env)) - (evalExpr(e2, env))
    }
    
    case Mult(e1, e2) => {
        (evalExpr(e1, env)) * (evalExpr(e2, env))
    }
    
    case Div(e1, e2) => {
        (evalExpr(e1, env)) / (evalExpr(e2, env))
    }
    
    case Exp(e) => math.exp( evalExpr(e, env))
    
    case Sine(e) => math.sin( evalExpr(e, env))
    
    case Cosine(e) => math.cos(evalExpr(e, env))
}

defined function evalExpr

In [12]:
// BEGIN TESTS
def checkSolution(e: Expr, v: Double): Boolean = {
    val y = evalExpr(e, Map{"x" -> v}) 
    math.abs(y) <= 1e-08
}

val e1 = Plus(Minus(Mult(Ident("x"), Ident("x")), Mult(Const(3.0), Ident("x"))), Const(2.0))
val v1 = solveEquation(e1, 10.0, 1000)

assert(checkSolution(e1, v1), s"Test 1 failed: $e1 == 0, your code returned $v1 with f(x) = ${evalExpr(e1, Map{"x" -> v1}) }")

// Sine(x)  - Cos(x) - x == 0
val x = Ident("x")
val e2 = Minus(Sine(x), Plus(Cosine(x), x))
val v2 = solveEquation(e2, 1.4)
assert(checkSolution(e2, v2), s"Test 2 failed: $e2 == 0, your code returned $v2 with f(x) = ${evalExpr(e2, Map{"x" -> v2}) }")

// e^x - 5.0 = 0
val e3 = Minus(Exp(x), Const(5.0))
val v3 = solveEquation(e3, 2.0)
assert(checkSolution(e3, v3), s"Test 3 failed: $e3 == 0, your code returned $v3 with f(x) = ${evalExpr(e3, Map{"x" -> v3}) }")

// e^cos(x) + e^sin(x) - 2.0 = 0

val e4 = Minus(Plus(Exp(Sine(x)), Exp(Cosine(x))), Const(2.0))
val v4 = solveEquation(e4, 1.8)
assert(checkSolution(e4, v4), s"Test 3 failed: $e4 == 0, your code returned $v4 with f(x) = ${evalExpr(e4, Map{"x" -> v4}) }")


// x sin(x) -  cos(x) - 5.0
val e5 = Minus(Mult(x, Sine(x)), Plus(Cosine(x), Const(5.0)))
val v5 = solveEquation(e5, 1.8)
assert(checkSolution(e5, v5), s"Test 3 failed: $e5 == 0, your code returned $v5 with f(x) = ${evalExpr(e5, Map{"x" -> v5}) }")

passed(10)
// END TESTS

Tests Passed (10 points)

defined function checkSolution
e1: Plus = Plus(
  Minus(Mult(Ident("x"), Ident("x")), Mult(Const(3.0), Ident("x"))),
  Const(2.0)
)
v1: Double = 2.0
x: Ident = Ident("x")
e2: Minus = Minus(Sine(Ident("x")), Plus(Cosine(Ident("x")), Ident("x")))
v2: Double = -1.2587281774926764
e3: Minus = Minus(Exp(Ident("x")), Const(5.0))
v3: Double = 1.6094379124341005
e4: Minus = Minus(
  Plus(Exp(Sine(Ident("x"))), Exp(Cosine(Ident("x")))),
  Const(2.0)
)
v4: Double = 2.6608333328389557
e5: Minus = Minus(
  Mult(Ident("x"), Sine(Ident("x"))),
  Plus(Cosine(Ident("x")), Const(5.0))
)
v5: Double = 21.80586857241091

## Problem 2: map, filter, reduce on containers (25 points)

Solve the problems using a combination of map, filter and foldLeft/foldRight opertions over lists. The use of mutables, recursion, For/While loops is forbidden for this problem.

In [11]:
/*
    x0 is the initial variable that you are going to keep updating if its not 
    in range of math.scala.abs(FUNCTION(current_x0)), make a helper equation 
    
    evaulate the initial expression, this will be f(x) 
    pass an iterator counter to keep track of iterations
    
    evalExpr, take in a map and and an expression, we only need to worry about mapping values to x
    now the only thing you really need to think about for evalExpr is that you need to 
    pass it an environmenet with the updated mapping of what x is currently, you will do this
    instead of using a var. 
        - step one, figure out how to create a map, then try to pass it to eval function 
        - step two, figure out how to update the mapping of "x" as it will need to be changed when 
        the condition is not satisfied. 
*/

// this function is essentially going to be replacing 'var' variables altogether
def updateEnvironment(current_env: Map[String,Double], nextValue: Double): Map[String,Double] = {
    val new_env = current_env + ("x" -> nextValue)  // this actually updates the mapping of x, changing its value 
    return new_env
}

// function is passed the value of the current f(x) has just evaluated
// and returns a boolean letting us know if have met the constraint or not 
def checkValue(x: Double): Boolean = {
    val value = (x).abs
    if ( value <= math.pow(10,-8)){
        true
    }else{
        false
    }
}

//def evalExpr (e: Expr, env: Map[String, Double]), 
// . this function will return a double that evaluate the current expression
//  with the current mapping of x 


// def derivativeHelper(e: Expr, x: String): Expr
// this function returns the expression of the derivative of the function. 

def solveHelper(e: Expr, x: Double, env: Map[String,Double], currentIter: Int = 0, maxIters:Int = 1000): Double = {
    if (currentIter == maxIters){
        env("x") // returns the last calculated value if hit 1000 calculations 
    }else{
        val value = env("x")  // give me the current value that we have calculated
        if(checkValue(value)){  // check that value if it meets criteria 
            value
        }else{
            // now we need to calculate the next value then keep calling this function 
            //  x_i+1 = x - f(x)/df(x)
            val f = e
            val df = derivativeExpr(e,"x") // gives us the derivative w.r.t x 
            
            val f_val = evalExpr(f,env)  
            val df_val = evalExpr(df,env)
            val prev_value = value
            
            // calculate the actual value for the newValue
            val calculatedValue = (prev_value - (f_val/df_val))
            // now update the map with this new value
            val next_environment = updateEnvironment(env, calculatedValue)
            val nextGuess = next_environment("x")
            
            // now proceed 
            solveHelper(e, nextGuess, next_environment, currentIter + 1, maxIters)
        }
    }
}
def solveEquation(e: Expr, x0: Double, maxIters:Int = 1000): Double = {
    val env = Map("x" -> x0)  // loading the initial guess 
    val initialValue = env("x")
    val answer = solveHelper(e,initialValue,env,0,1000)
    answer
    
}
    

defined function updateEnvironment
defined function checkValue
defined function solveHelper
defined function solveEquation


### 2A: Count how many whitespaces in a string (5 points)

Write a function `countWhiteSpaces` that will count how many characters in a string are white spaces ` ` or `\t`.

In [13]:
def countWhiteSpaces(s: String): Int ={
    val count = s.filter(x => x == ' ' || x == "\t")
    count.length
}
    

defined function countWhiteSpaces

In [14]:
// BEGIN TESTS
val t1  = countWhiteSpaces("Hello")
assert(t1 == 0, s"Test1: String whould have 0 white spaces. Your code returned $t1")
val t2 = countWhiteSpaces("Hello World this is Sriram Speaking")
assert(t2 ==5, "Test 2: String should have 5 white spaces.Your code returned $t2 ")
val t3 = countWhiteSpaces("No Non Nons Nonse Nonsen Nonsens Nonsense!!")
assert(t3 ==6, "Test 3")
val t4 = countWhiteSpaces("      ")
assert(t4==6, s"Test 4: String  (6spaces) should have  white spaces.Your code returned $t4 ")

passed(5)
// END TESTS

Tests Passed (5 points)

t1: Int = 0
t2: Int = 5
t3: Int = 6
t4: Int = 6

### 2B: Sum of Even  Elements Minus Sum of Odd Elements (5 points)

Given a list of length n, write a function that returns the sum of even elements minus sum of odd elements of a list. It must return $0$ for the empty list.

eg., input `List(1, 3, 5, 4, 5, 2, 1, 0)`. Output `  (4 + 2 + 0) - (1 + 3 + 5 + 5 + 1) = -9`

In [15]:
def sumOfEvenMinusOdd(l: List[Int]) : Int ={
    if (l.isEmpty){
        0
    }else{
        val evenElements = l.filter( x => (x).abs % 2 == 0).foldLeft(0)((acc,x) => acc + x)
        val oddElements = l.filter( x => (x).abs % 2 == 1).foldLeft(0)((acc,x) => acc + x)
        evenElements - oddElements
    }
}

defined function sumOfEvenMinusOdd

In [16]:
// BEGIN TESTS
val i1 = sumOfEvenMinusOdd(List(1,3,5,4,5,2,1,0))
assert(i1 == -9, "Test 1 Failed")

val i2 = sumOfEvenMinusOdd(List(2,4,5,6,7,8,10))
assert(i2 == 18, "Test 2 Failed")

val i3 = sumOfEvenMinusOdd(List(109, 19, 12, 1, -5, -120, -15, 30,-33,-13, 12, 19, 3, 18, 1, -1))
assert(i3 == -133, "Test 3 Failed")

passed(5)
// END TESTS

Tests Passed (5 points)

i1: Int = -9
i2: Int = 18
i3: Int = -133

### 2C: sum of odd index elements - sum of even index elements in a list (5 points)

Given a list $L= [a(1), ..., a(n)]$ of $n$ integers, find the  the sum
of elements at even positions (a[0], a[2], ..., ) minus the sum of elements at
odd positions (a[1], a[3], ... ) :
(a[0] + a[2]+a[4] + .... ) - (a[1] + a[3]  + .... )



In [17]:
def sumOfEvenIndicesMinusOdd(l: List[Int]): Int = {
    // just check if the list is empty first
    if (l.isEmpty){
        0
    }
  
    if (l.length == 2){
        l(0) - l(1)
    }
    val evenSum = l.foldLeft(0)((acc,currentItem) => {
        if (l.slice(l.indexOf(currentItem),l.size).size % 2 == 0){
            acc + currentItem
        }else{
            acc
        }
    })
    
    val oddSum = l.foldLeft(0)((acc,currentItem) => {
        if (l.slice(l.indexOf(currentItem),l.size).size % 2 == 1){
            acc + currentItem
        }else{
            acc
        }
    })
 
    println("even sum, ", evenSum)
    println("odd sum, ", oddSum)
    evenSum - oddSum
    
    
         
} 

defined function sumOfEvenIndicesMinusOdd

In [18]:
// BEGIN TESTS
val t1 = sumOfEvenIndicesMinusOdd(List(1, 2, 3, 4, 5, 6))
assert(t1 == -3, "Test 1 failed")
val t2 = sumOfEvenIndicesMinusOdd(List(10, 200))
assert(t2 == -190, "Test 2 failed")
val t3 = sumOfEvenIndicesMinusOdd(List(0, 20, 0, 20, 0, 20, 20, 0, 20, 0, 20, 0))
assert(t3 == 0, "Test 3 failed")
val t4 = sumOfEvenIndicesMinusOdd(List())
assert(t4 == 0, "Test 4 failed")
val t5 = sumOfEvenIndicesMinusOdd(List(10))
assert(t5 == 10, "Test 5 failed")

passed(5)
// END TESTS

(even sum, ,9)
(odd sum, ,12)
(even sum, ,10)
(odd sum, ,200)
(even sum, ,0)
(odd sum, ,120)


: 

### 2D: Loop → FoldLeft (10 points)
Eliminate the for loop and mutable variables in the code below

In [1]:
def imperativeFun(lst: List[Int]): Int = {
    var minSoFar = Int.MaxValue // 2^32-1
    var maxDiff = 0
    for (x <- lst) {
        if (x <= minSoFar){
            minSoFar = x
        }
        if (x - minSoFar > maxDiff){
            maxDiff = x - minSoFar
        }
    }
    
    return maxDiff
}

defined function imperativeFun

In [59]:
//3 2 1 0
def funUsingFoldLeft(lst:List[Int]): Int = {
    if (lst.size < 2){
        0
    }else{
            val (maxDiff, minSoFar) =  lst.foldLeft ((0, Int.MaxValue)) { // acc is a pair of Ints. First of the pair is "x" and second is "y" but they are not vars anymore.
                    case ( tuple: (Int, Int), currentElement: Int ) => {
                           val maxDiff = tuple._1 // Unpack the acc
                           val minSoFar = tuple._2 

                           if (currentElement <= minSoFar) { // setting a new minimum 
                              val newMinSoFar = currentElement
                              val newDiffCalculated = currentElement - newMinSoFar
                              if (newDiffCalculated > maxDiff){
                                  (newDiffCalculated,newMinSoFar)
                              }else{
                                  (maxDiff,newMinSoFar)
                              }
                           } 
                           else {
                               if (currentElement - minSoFar > maxDiff){
                                   val newDiffCalculated = currentElement - minSoFar
                                   (newDiffCalculated, minSoFar)
                               }
                               else{
                                   (maxDiff,minSoFar)
                               }
                           }
                    }
                  }
        println(">>", maxDiff)
        maxDiff
    }
    
    
    
}

defined function funUsingFoldLeft

In [60]:
// BEGIN TESTS
val l1 = List(1, 2, 3, -1, 2, 5, 18, 19, 1, 0, -12, 2)
assert(imperativeFun(l1) == funUsingFoldLeft(l1), "Test 1 failed")

val l2 = List(2)
assert(imperativeFun(l2) == funUsingFoldLeft(l2), "Test 2 failed")

val l3 = List()
assert(imperativeFun(l3) == funUsingFoldLeft(l3), "Test 3 failed")

val l4 = List(3, 2, 1, 0)
assert(imperativeFun(l4) == funUsingFoldLeft(l4), "Test 4 failed")

val l5 = List( -1,  -12, -22, -54, -89, 90)
assert(imperativeFun(l5) == funUsingFoldLeft(l5), "Test 5 failed")

passed(10)
// END TESTS

(>>,20)
(>>,0)
(>>,179)
Tests Passed (10 points)

l1: List[Int] = List(1, 2, 3, -1, 2, 5, 18, 19, 1, 0, -12, 2)
l2: List[Int] = List(2)
l3: List[Nothing] = List()
l4: List[Int] = List(3, 2, 1, 0)
l5: List[Int] = List(-1, -12, -22, -54, -89, 90)

defined function funUsingFoldLeft